In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Convert Labelme JSON (rectangle) to Ultralytics YOLO detection txt.
Assumptions:
- class "swd" -> id 0, "p" -> id 1 (modify CLASS_TO_ID if you add more classes)
- Only rectangles are used for detection (points are ignored)
"""

from pathlib import Path
import json, argparse

CLASS_TO_ID = {"p": 0}   # add more classes if needed

def norm(val, denom): return float(val) / float(denom)

def rect_from_points(pts):
    xs = [p[0] for p in pts]; ys = [p[1] for p in pts]
    x1, x2 = min(xs), max(xs); y1, y2 = min(ys), max(ys)
    return x1, y1, x2, y2

def convert_one(json_path: Path, out_dir: Path):
    data = json.loads(Path(json_path).read_text())
    W, H = data["imageWidth"], data["imageHeight"]

    # collect rectangles only (ignore points for detection)
    rects = []
    for sh in data["shapes"]:
        t = sh.get("shape_type", "")
        lbl = sh.get("label", "")
        pts = sh.get("points", [])
        if t == "rectangle" and lbl in CLASS_TO_ID:
            rects.append({"label": lbl, "xyxy": rect_from_points(pts)})

    # if no rectangles, skip
    if not rects:
        return

    # write yolo txt (detection format)
    out_dir.mkdir(parents=True, exist_ok=True)
    stem = Path(data["imagePath"]).stem
    out_txt = out_dir / f"{stem}.txt"

    lines = []
    for r in rects:
        x1, y1, x2, y2 = r["xyxy"]
        cx = (x1 + x2) / 2.0; cy = (y1 + y2) / 2.0
        w = (x2 - x1); h = (y2 - y1)
        cxn, cyn, wn, hn = norm(cx, W), norm(cy, H), norm(w, W), norm(h, H)

        cls_id = CLASS_TO_ID[r["label"]]
        # Detection format: class_id center_x center_y width height
        line = f"{cls_id} {cxn:.6f} {cyn:.6f} {wn:.6f} {hn:.6f}"
        lines.append(line)

    out_txt.write_text("\n".join(lines))

if __name__ == "__main__":
    ann_dir = Path("/workspace/models/runs_yolov11_det/cropped_objects/swd")
    out_dir = Path("cropped_objects")
    
    # 确保输出目录存在
    out_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"Converting Labelme JSON files from: {ann_dir}")
    print(f"Output directory: {out_dir}")
    
    converted_count = 0
    for jp in ann_dir.glob("*.json"):
        try:
            convert_one(jp, out_dir)
            converted_count += 1
            print(f"✓ Converted: {jp.name}")
        except Exception as e:
            print(f"[WARN] Failed to convert {jp.name}: {e}")
    
    print(f"\nConversion completed. Converted {converted_count} files.")

    # 验证转换结果
    txt_files = list(out_dir.glob("*.txt"))
    print(f"Generated {len(txt_files)} annotation files in {out_dir}")
    
    if txt_files:
        # 显示第一个文件的内容作为示例
        sample_file = txt_files[0]
        print(f"\nSample annotation content from {sample_file.name}:")
        print(sample_file.read_text())

Converting Labelme JSON files from: /workspace/models/runs_yolov11_det/cropped_objects/swd
Output directory: cropped_objects
✓ Converted: 0607_0731_700_obj0_swd_uuid_ce655b7d-7c8b-4a1e-a863-32a000d82d0d.json
✓ Converted: 0607_0731_700_obj6_swd_uuid_35af4292-800d-4e97-91e4-eca5e5229ec9.json
✓ Converted: 0607_0801_700_obj1_swd_uuid_50a739e6-61d0-4b8b-aaeb-a692c56aa56e.json
✓ Converted: 0607_0831_700_obj4_swd_uuid_43850137-a234-44c7-8005-fc9f3ba5394d.json
✓ Converted: 0607_0901_700_obj2_swd_uuid_89efe46d-12bf-48a8-9e6e-aa57541b9df0.json
✓ Converted: 0607_1031_700_obj4_swd_uuid_4363ce35-f202-4d68-a674-1b6f0e941ef8.json
✓ Converted: 0607_1031_700_obj7_swd_uuid_68e56662-0412-4703-ab87-2b36a4cb2393.json
✓ Converted: 0611_1131_700_obj0_swd_uuid_d171f9a1-1d37-4230-893f-82049d74cb05.json
✓ Converted: 0614_1131_700_obj78_swd_uuid_78b859a6-390c-41a1-a975-607ec4ccff0d.json
✓ Converted: 0617_0947_780_obj21_swd_uuid_24bc624c-5c0a-429b-a6dd-6ccfa6f4ab64.json
✓ Converted: 0617_1208_780_obj22_swd_uuid_0